In [43]:
from google.colab import files
files.upload()

KeyboardInterrupt: ignored

In [1]:
import numpy as np
import re
import shutil
import os
import unicodedata
import urllib3
import zipfile
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [41]:
import pandas as pd

sents = pd.read_csv('1000sents.csv', error_bad_lines=False)

b'Skipping line 676: expected 12 fields, saw 13\nSkipping line 764: expected 12 fields, saw 13\n'


In [46]:
sents = sents[['HEADWORD','JAPANESE','EXAMPLE (KO)','EXAMPLE (JA)']]

In [48]:
kr, jp = sents['EXAMPLE (KO)'], sents['EXAMPLE (JA)']

In [49]:
sents.drop(columns=['EXAMPLE (KO)','EXAMPLE (JA)'], inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [58]:
a = pd.concat([kr,jp], axis=1)
a.columns = ['HEADWORD', 'JAPANESE']

In [60]:
sents = pd.concat([sents, a], axis=0)
sents.columns = ['eng','jp']

In [61]:
sents

,eng,jp
0,가게,店
1,가격,"価格, 値段"
2,가깝다,近い
3,가끔,たまに
4,가능하다,"可能だ, できる"
...,...,...
993,휴지는 휴지통에 버리세요.,紙くずはごみ箱に捨ててください。
994,우리 집 근처에 작은 강이 흘러요.,我家の近くに小さな川が流れてます。
995,흰색이 니 까만 피부랑 잘 어울려.,白がお前の黒い肌によく似合う。
996,오늘은 테니스 칠 힘이 없어.,今日はテニスやる力がない。


In [62]:
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFC', s)
      if unicodedata.category(c) != 'Mn')

In [64]:
def preprocess_sentence(sent):
    # 위에서 구현한 함수를 내부적으로 호출
    sent = unicode_to_ascii(sent.lower())

    # 단어와 구두점 사이에 공백을 만듭니다.
    # Ex) "he is a boy." => "he is a boy ."
    sent = re.sub(r"([?.!,¿])", r" \1", sent)

    # (a-z, A-Z, ".", "?", "!", ",") 이들을 제외하고는 전부 공백으로 변환합니다.
    sent = re.sub(r"[^가-힇ㄱ-ㅎㅏ-ㅣぁ-ゔァ-ヴー々〆〤一-龥!.?。]+", r" ", sent)

    sent = re.sub(r"\s+", " ", sent)
    return sent

In [65]:
# 전처리 테스트
en_sent = u"저녁 먹었어요?"
jp_sent = u"ご飯食べたか。"
print(preprocess_sentence(en_sent))
print(preprocess_sentence(jp_sent))

저녁 먹었어요 ?
ご飯食べたか。


In [12]:
!pip install sentencepiece==0.1.83

     |████████████████████████████████| 1.0MB 11.2MB/s 


In [66]:
with open('english.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(sents['eng']))

In [67]:
import sentencepiece as spm
spm.SentencePieceTrainer.Train('--input=english.txt --model_prefix=english_train --vocab_size=32000 --model_type=unigram --max_sentence_length=9999 --character_coverage=1.0 --hard_vocab_limit=false')

True

In [68]:
import sentencepiece as spm
sp_en = spm.SentencePieceProcessor()
vocab_file = "english_train.model"
sp_en.load(vocab_file)

True

In [69]:
with open('japanese.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(sents['jp']))

In [70]:
spm.SentencePieceTrainer.Train('--input=japanese.txt --model_prefix=japanese_train --vocab_size=32000 --model_type=unigram --character_coverage=0.9995 --max_sentence_length=9999 --hard_vocab_limit=false')

True

In [71]:
sp_jp = spm.SentencePieceProcessor()
vocab_file = "japanese_train.model"
sp_jp.load(vocab_file)

True

In [72]:
sp_en.get_piece_size(), sp_jp.get_piece_size()

(2225, 3509)

In [20]:
# START_TOKEN, END_TOKEN = [sp_en.get_piece_size()], [sp_en.get_piece_size() + 1]
# START_TOKEN_J, END_TOKEN_J = [sp_jp.get_piece_size()], [sp_jp.get_piece_size() + 1]
# # 시작 토큰과 종료 토큰을 고려하여 단어 집합의 크기를 + 2
# VOCAB_SIZE = sp_en.get_piece_size() + 2
# VOCAB_SIZE_J= sp_jp.get_piece_size() + 2

In [73]:
num_samples = 5000

In [74]:
def load_preprocessed_data():
    encoder_input, decoder_input, decoder_target = [], [], []
    i = 0
    sos = [1]
    eos = [2]
    for kr, jp in zip(sents['eng'], sents['jp']) :
        src_line, tar_line = kr, jp
        src_line_input = [w for w in sp_en.EncodeAsIds(preprocess_sentence(src_line))]

        # # target 데이터 전처리
        tar_line_input = sos + (sp_jp.EncodeAsIds(preprocess_sentence(tar_line)))
        tar_line_target = (sp_jp.EncodeAsIds(preprocess_sentence(tar_line))) + eos
        encoder_input.append(src_line_input)
        decoder_input.append(tar_line_input)
        decoder_target.append(tar_line_target)
        i += 1
        if i == num_samples - 1:
                break
    return encoder_input, decoder_input, decoder_target

In [75]:
encoder_input, decoder_input, decoder_target = load_preprocessed_data()
print(encoder_input[:5])
print(decoder_input[:5])
print(decoder_target[:5])

[[160], [747], [983], [1290], [1186]]
[[1, 1610], [1, 1566, 3, 483], [1, 3, 1403], [1, 3, 1535], [1, 3, 1109, 844, 14, 1630]]
[[1610, 2], [1566, 3, 483, 2], [3, 1403, 2], [3, 1535, 2], [3, 1109, 844, 14, 1630, 2]]


In [76]:
encoder_input = pad_sequences(encoder_input, padding="post")
decoder_input = pad_sequences(decoder_input, padding="post")
decoder_target = pad_sequences(decoder_target, padding="post")

In [77]:
src_vocab_size = sp_en.get_piece_size() + 1
tar_vocab_size = sp_jp.get_piece_size() + 1
print("영어 단어 집합의 크기 : {:d}, 일어 단어 집합의 크기 : {:d}".format(src_vocab_size, tar_vocab_size))

영어 단어 집합의 크기 : 2226, 일어 단어 집합의 크기 : 3510


In [78]:
indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)
print(indices)

[ 981  957  149 ... 1063 1081 1509]


In [79]:
encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]

In [80]:
encoder_input.shape
decoder_input.shape

(1996, 20)

In [81]:
n_of_val = int(2000*0.1)
print(n_of_val)

200


In [82]:
encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

In [83]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking
from tensorflow.keras.models import Model

In [84]:
latent_dim = 50

In [85]:
# 인코더
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(src_vocab_size, latent_dim)(encoder_inputs) # 임베딩 층
enc_masking = Masking(mask_value=0.0)(enc_emb) # 패딩 0은 연산에서 제외
encoder_lstm = LSTM(latent_dim, return_state=True) # 상태값 리턴을 위해 return_state는 True
encoder_outputs, state_h, state_c = encoder_lstm(enc_masking) # 은닉 상태와 셀 상태를 리턴
encoder_states = [state_h, state_c] # 인코더의 은닉 상태와 셀 상태를 저장

In [86]:
# 디코더
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(tar_vocab_size, latent_dim) # 임베딩 층
dec_emb = dec_emb_layer(decoder_inputs) # 패딩 0은 연산에서 제외
dec_masking = Masking(mask_value=0.0)(dec_emb)

# 상태값 리턴을 위해 return_state는 True, 모든 시점에 대해서 단어를 예측하기 위해 return_sequences는 True
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True) 

# 인코더의 은닉 상태를 초기 은닉 상태(initial_state)로 사용
decoder_outputs, _, _ = decoder_lstm(dec_masking,
                                     initial_state=encoder_states)

# 모든 시점의 결과에 대해서 소프트맥스 함수를 사용한 출력층을 통해 단어 예측
decoder_dense = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [87]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [88]:
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [89]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 50)     111300      input_5[0][0]                    
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, None, 50)     175500      input_6[0][0]                    
____________________________________________________________________________________________

In [90]:
import datetime
import keras
from keras.callbacks import EarlyStopping

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
checkpoint_filepath = "seq_best.hdf5"
save_best = keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=True, mode='auto', save_freq='epoch', options=None)
early_stopping = EarlyStopping(monitor='val_acc', mode='min', verbose=1, patience=10)

In [39]:
# !pip install wandb

In [40]:
# !wandb login eb738ba304b560fc8b41bf28ba686d71a2b3d5d4

In [41]:
# import wandb
# from wandb.keras import WandbCallback

In [42]:
# wandb_project = "seq2"
# wandb_group = ""

In [91]:
#wandb.init(project=wandb_project) 

#wandb.config.epochs = 50
#wandb.config.batch_size = 16


model.fit(x = [encoder_input_train, decoder_input_train], y = decoder_target_train, \
          validation_data = ([encoder_input_test, decoder_input_test], decoder_target_test),
          batch_size =24, epochs = 200, callbacks=[save_best])

Epoch 1/200
75/75 [==============================] - 12s 52ms/step - loss: 5.8966 - acc: 0.6643 - val_loss: 2.3525 - val_acc: 0.7010

Epoch 00001: val_acc improved from -inf to 0.70100, saving model to seq_best.hdf5
Epoch 2/200
75/75 [==============================] - 1s 16ms/step - loss: 2.0965 - acc: 0.7093 - val_loss: 1.9423 - val_acc: 0.7050

Epoch 00002: val_acc improved from 0.70100 to 0.70500, saving model to seq_best.hdf5
Epoch 3/200
75/75 [==============================] - 1s 17ms/step - loss: 1.7444 - acc: 0.7286 - val_loss: 1.8232 - val_acc: 0.7542

Epoch 00003: val_acc improved from 0.70500 to 0.75425, saving model to seq_best.hdf5
Epoch 4/200
75/75 [==============================] - 1s 16ms/step - loss: 1.6437 - acc: 0.7702 - val_loss: 1.7907 - val_acc: 0.7682

Epoch 00004: val_acc improved from 0.75425 to 0.76825, saving model to seq_best.hdf5
Epoch 5/200
75/75 [==============================] - 1s 16ms/step - loss: 1.5512 - acc: 0.7831 - val_loss: 1.8224 - val_acc: 0.774

In [92]:
# 인코더
encoder_model = Model(encoder_inputs, encoder_states)

In [93]:
# 디코더
# 이전 시점의 상태를 보관할 텐서
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# 훈련 때 사용했던 임베딩 층을 재사용
dec_emb2= dec_emb_layer(decoder_inputs)

# 다음 단어 예측을 위해 이전 시점의 상태를 현 시점의 초기 상태로 사용
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]

# 모든 시점에 대해서 단어 예측
decoder_outputs2 = decoder_dense(decoder_outputs2)

In [94]:
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [95]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <SOS>에 해당하는 정수 생성
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = sp_jp.piece_to_id('<s>')

    stop_condition = False
    decoded_sentence = ''

    # stop_condition이 True가 될 때까지 루프 반복
    # 구현의 간소화를 위해서 이 함수는 배치 크기를 1로 가정합니다.
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        # 예측 결과를 단어로 변환
        sampled_token_index = int(np.argmax(output_tokens[0, -1, :]))
        sampled_char = sp_jp.IdToPiece(sampled_token_index)

         # 현재 시점의 예측 단어를 예측 문장에 추가
        decoded_sentence += ' '+sampled_char

        # <eos>에 도달하거나 정해진 길이를 넘으면 중단.
        if (sampled_char == '</s>' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

In [96]:
# 원문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2src(input_seq):
    temp=''
    for i in input_seq:
        if(i!=0):
            temp = temp + sp_en.id_to_piece(int(i))+' '
    return temp

# 번역문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2tar(input_seq):
    temp=''
    for i in input_seq:
        if((i!=0 and i!=sp_jp.piece_to_id('<s>')) and i!=sp_jp.piece_to_id('</s>')):
            temp = temp +  sp_jp.id_to_piece(int(i)) + ' '
    return temp

In [97]:
for seq_index in [3,50,100,300,1001]:
  input_seq = encoder_input_train[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)

  print("원문 : ",seq2src(encoder_input_train[seq_index]))
  print("번역문 :",seq2tar(decoder_input_train[seq_index]))
  print("예측문 :",decoded_sentence[:-5])
  print("\n")

원문 :  ▁가깝다 
번역문 : ▁ 近い 
예측문 :  ▁ サッカー


원문 :  ▁가지 
번역문 : ▁ 種類 
예측문 :  ▁ 種類


원문 :  ▁우리 는 ▁어 렸 을 ▁때 ▁같은 ▁학교에 ▁ 다 녔 다 ▁ . 
번역문 : ▁私たちは 幼い頃 同じ 学校に 通 った 。 
예측문 :  ▁私たちは 幼い頃 同じ 学校に 通 った 。


원문 :  ▁우리 는 ▁밤새 ▁춤 추 고 ▁노래 했다 ▁ . 
번역문 : ▁私たちは 一晩中 歌 って 踊 った 。 
예측문 :  ▁ 一晩中 に お仕舞い に ▁彼は なる だ 。


원문 :  ▁복잡하 다 
번역문 : ▁ 複雑 だ 
예측문 :  ▁ ない ▁ 個




In [99]:
for seq_index in [1200,1100,77,49,999]:
  input_seq = encoder_input_train[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)

  print("원문 : ",seq2src(encoder_input_train[seq_index]))
  print("번역문 :",seq2tar(decoder_input_train[seq_index]))
  print("예측문 :",decoded_sentence[:-5])
  print("\n")

원문 :  ▁피우 다 
번역문 : ▁ タバコを ▁ 吸う 
예측문 :  ▁ 入れる


원문 :  ▁아마 
번역문 : ▁多分 
예측문 :  ▁一緒に


원문 :  ▁우리 ▁회사 에는 ▁일하 는 ▁사람이 ▁적다 ▁ . 
번역문 : ▁当 社 には 働き 手 が 少ない 。 
예측문 :  ▁当 社 には 働き 手 が 少ない 。


원문 :  ▁찾 다 
번역문 : ▁探 す ▁ 見つけ る 
예측문 :  ▁もう ▁ 既に


원문 :  ▁며칠 
번역문 : ▁ 何日 ▁ 数日 
예측문 :  ▁ ▁ 数日


